In [ ]:
import json
import numpy as np
import pandas as pd
import requests
import sqlite3
import yaml
from requests.exceptions import HTTPError

Read configuration file

In [ ]:
with open(r'../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

url_discogs_api = "https://api.discogs.com"

# Retrieve the collection items

Get number of pages of collection items

In [ ]:
try:
    query = {'page': 1, 'per_page': 100}
    url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
    response = requests.get(url_request, params=query)
    response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

no_pages = jsonResponse["pagination"]["pages"]

Get the collection items

In [ ]:
collection_jsons = []
for i in range(1, no_pages + 1):
    query = {'page': i, 'per_page': 100}
    url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
    response = requests.get(url_request, params=query)
    jsonResponse = response.json()
    collection_jsons.append(jsonResponse["releases"])

In [ ]:
collection_items = []
for i in range(0, len(collection_jsons)):
    collection_items.append(pd.json_normalize(collection_jsons[i])) 

df_collection = pd.concat(collection_items)

In [ ]:
df_collection.head()

# Get marketplace values

In [ ]:
try:
    query = {'curr_abbr': 'EUR'}
    url_request = url_discogs_api + "/marketplace/stats/" + "403287"
    response = requests.get(url_request, params=query)
    response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

pd.json_normalize(jsonResponse)